### Создаем витрины по осищенным данным по расмещенным и выкупленным товарам

In [1]:
import numpy as np 
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt 
from PIL import Image

### Комментарии !!!!!!!!!!!

Данные необходимо смотреть: 

- По всему срезу данных:

Без фильтра по колонке "Status" (только при чистке данных удаляем строки, где "Status" == 'Отменен'. Так как заказы, которые не оформили и мы их не считаем за рассматриваемые и не выкупленные товары: 'Отменен'). 

- По выкупленным товарам:

По выкупленным товарам нужно сделать фильтр по колонке "Status". Значение должно быть равно - 'Выдан клиенту', тогда товар может считаться выкупленным (при чистке данных удаляем строки, где "Status" == 'Отменен'). 

- По чекам с идентификатором клиента:

За идентификатор клиента я беру номер телефона Phone_new. Так же, можно взять Email_new (но телефон являются лучшим способом). После построения процентного содержания значений NaN в слобцах Phone_new и Email_new, мы видим, что значений NaN в этих столбцах нет. Это значит, что все чеки по всему очищенному (!) срезу данных являются чеками с идентификатором клиента. 

#### Выручка | Абсолютная маржа | Маржа | Средняя маржа | Средний чек | Кол-во чеков | Кол-во товаров | Среднее кол-во товаров в  чеке | Кол-во уникальных клиентов | Ценность клиента

- #### МесяцДатыЗаказа
- #### Группа2
- #### Регион        -- 599 уникальных значений
- #### ГородМагазина -- 64 уникальных значений
- #### МагазинЗаказа -- 89 уникальных значений
- #### Гео           -- 'Москва', 'МО', 'Регионы'
- #### МетодДоставки 
- #### ФормаОплаты
- #### Телефон_new (or ЭлектроннаяПочта_new)
- #### ДеньНедели
- #### ВремяОкругл

In [2]:
def generate_table(df, column_name = "МесяцДатыЗаказа"): 
    columns_list = [column_name, 'Выручка', 'Абсолютная маржа', 'Маржа', 'Средняя маржа',
                                 'Средний чек', 'Кол-во чеков', 'Кол-во товаров', 
                                 'Среднее кол-во товаров в  чеке',
                                 'Кол-во уникальных клиентов', 'Ценность клиента']
    
    unique_values = df[column_name].unique()
    total = pd.DataFrame(columns=columns_list)
    
    for val in unique_values:
        
        list_of_values = [val] 
        filtered_df = df[df[column_name] == val]
        
        # Подсчет значений
        
        # Выручка
        profit = int(round(filtered_df['Нов_СуммаСтроки'].sum()))
        list_of_values.append(profit)
        
        # Абсолютная маржа
        absmargin = int(round(filtered_df['Нов_Маржа'].sum()))
        list_of_values.append(absmargin)
        
        # Маржа
        margin = absmargin/profit                                                
        list_of_values.append(margin)
        
        # Средняя маржа !!!
        df_numb_check = filtered_df['НомерЗаказаНаСайте'].unique() # !!!
        numb_check = df_numb_check.size                            # !!!
        absmargin = int(round(filtered_df['Нов_Маржа'].sum()))
        list_of_values.append(absmargin/numb_check) 
        
        # Средний чек
        avarage_bill = int(round(
            filtered_df.groupby(['НомерЗаказаНаСайте']).mean()['Нов_СуммаЗаказаНаСайте'].mean()))
        list_of_values.append(avarage_bill)
        
        # Кол-во чеков !!!
        list_of_values.append(numb_check)
        
        # Кол-во товаров
        numb_good = filtered_df['МесяцДатыЗаказа'].size
        list_of_values.append(numb_good) 
        
        # Среднее кол-во товаров в чеке
        avarage_numb_good = int(round(
            filtered_df.groupby(['НомерЗаказаНаСайте']).count()['Нов_СуммаЗаказаНаСайте'].mean()))
        list_of_values.append(avarage_numb_good)
        
        # Кол-во уникальных клиентов
        num_of_customers = filtered_df['Телефон_new'].unique().size
        list_of_values.append(num_of_customers)

        # Ценность клиента
        value_of_customer = int(round(absmargin /num_of_customers))
        list_of_values.append(value_of_customer) 
        
        tmp_df = pd.DataFrame([list_of_values], columns=columns_list) 
        total = total.append(tmp_df, ignore_index=True)
        
    return total

### Читаем датасет и добавляем дни недели и время, когда был размещен заказ

In [92]:
df_all = pd.read_csv('for_tables_filtered_data_Размещенные.csv', sep=';')

In [93]:
# День недели
dates = pd.to_datetime(df_all['Дата'])
df_all['Дата_дата'] = dates.dt.date
df_all['ДеньНеделиЦифра'] = dates.dt.dayofweek
df_all['ДеньНедели'] = dates.dt.weekday_name

# Время - час
df_all['ВремяОкругл'] = dates.dt.hour

### Переименуем дни недели на английском на русский

In [94]:
df_all.loc[df_all['ДеньНедели'] == 'Monday', 'ДеньНедели'] = 'Понедельник'
df_all.loc[df_all['ДеньНедели'] == 'Tuesday', 'ДеньНедели'] = 'Вторник'
df_all.loc[df_all['ДеньНедели'] == 'Wednesday', 'ДеньНедели'] = 'Среда'
df_all.loc[df_all['ДеньНедели'] == 'Thursday', 'ДеньНедели'] = 'Четверг'
df_all.loc[df_all['ДеньНедели'] == 'Friday', 'ДеньНедели'] = 'Пятница'
df_all.loc[df_all['ДеньНедели'] == 'Saturday', 'ДеньНедели'] = 'Суббота'
df_all.loc[df_all['ДеньНедели'] == 'Sunday', 'ДеньНедели'] = 'Воскресенье'

In [95]:
df_all.shape

(908322, 45)

In [96]:
df_all.to_csv('for_tables_filtered_data_Размещенные.csv', index=False, sep=';', encoding='utf-8-sig')

### Фильтры для датасета по всему срезу данных. 

In [97]:
df_all = df_all.dropna(subset = ['Группа2'])
df_all = df_all.dropna(subset = ['МетодДоставки'])
df_all = df_all.dropna(subset = ['ФормаОплаты'])

In [98]:
df_all.shape

(908322, 45)

### Pivot tables 

In [99]:
pivot_table_Month_all = generate_table(df_all, column_name="МесяцДатыЗаказа") 
pivot_table_Month_all.sort_values('Выручка', ascending = False) 

,МесяцДатыЗаказа,Выручка,Абсолютная маржа,Маржа,Средняя маржа,Средний чек,Кол-во чеков,Кол-во товаров,Среднее кол-во товаров в чеке,Кол-во уникальных клиентов,Ценность клиента
0,5,210350562,68756796,0.326868,1002.811913,3352,68564,233971,3,54041,1272
3,8,206836428,60015180,0.290158,912.681235,3475,65757,247120,4,54777,1096
2,7,197767141,61246170,0.309688,998.291307,3541,61351,225533,4,50366,1216
1,6,181828009,56876389,0.312803,992.053112,3467,57332,201698,4,47322,1202


In [100]:
pivot_table_Group2_all = generate_table(df_all, column_name="Группа2") 
pivot_table_Group2_all.sort_values('Выручка', ascending = False)

,Группа2,Выручка,Абсолютная маржа,Маржа,Средняя маржа,Средний чек,Кол-во чеков,Кол-во товаров,Среднее кол-во товаров в чеке,Кол-во уникальных клиентов,Ценность клиента
0,КРУПНОГАБАРИТНЫЙ ТОВАР,268703186,75690061,0.281687,1812.110920,7481,41769,49599,1,36990,2046
9,ПОДГУЗНИКИ,124484671,41334394,0.332044,655.986955,3361,63011,89464,1,39878,1037
3,ИГРУШКИ,120002647,23696952,0.197470,313.236292,2832,75652,169143,2,61548,385
1,"ТЕКСТИЛЬ, ТРИКОТАЖ",104205715,30354567,0.291295,516.058603,3389,58820,181191,3,41395,733
10,ДЕТСКОЕ ПИТАНИЕ,74343443,48534703,0.652844,1253.608405,3611,38716,179392,5,23494,2066
2,ОБУВЬ,31019361,8517036,0.274572,408.804646,3147,20834,32496,2,16863,505
5,ТОВАРЫ ДЛЯ КОРМЛЕНИЯ,30378232,5761995,0.189675,207.504862,3697,27768,52988,2,22706,254
8,КОСМЕТИКА/ГИГИЕНА,22802113,6562713,0.287812,202.477879,3584,32412,93003,3,23940,274
4,"КАНЦТОВАРЫ, КНИГИ, ДИСКИ",12481950,3968956,0.317976,307.075899,3137,12925,42938,3,11272,352
12,ТОВАРЫ ДЛЯ ЖИВОТНЫХ,7244044,2078737,0.286958,480.743987,3124,4324,12578,3,3066,678


In [101]:
pivot_table_CityShop_all = generate_table(df_all, column_name="ГородМагазина") 
pivot_table_CityShop_all.sort_values('Выручка', ascending = False) 

,ГородМагазина,Выручка,Абсолютная маржа,Маржа,Средняя маржа,Средний чек,Кол-во чеков,Кол-во товаров,Среднее кол-во товаров в чеке,Кол-во уникальных клиентов,Ценность клиента
0,ИнтернетДоставка,740893034,230045264,0.310497,928.965352,3293,247636,902075,4,158665,1450
5,Москва,6266555,2173781,0.346886,3019.140278,9551,720,878,1,684,3178
12,Калуга,4518285,1287362,0.284923,3109.570048,11015,414,475,1,328,3925
2,Ростов-на-Дону,3417347,882435,0.258222,2855.776699,11494,309,358,1,223,3957
1,Омск,2795545,794825,0.284318,3629.337900,13836,219,266,1,200,3974
...,...,...,...,...,...,...,...,...,...,...,...
54,Сергиев,194690,63386,0.325574,2641.083333,9062,24,26,1,24,2641
34,Балаково,142590,46681,0.327379,3112.066667,9506,15,19,1,14,3334
61,Казань,133478,36042,0.270022,1802.100000,6674,20,20,1,19,1897
58,Реутов,105571,31932,0.302469,3193.200000,10557,10,13,1,10,3193


In [102]:
pivot_table_Geo_all = generate_table(df_all, column_name="Гео") 
pivot_table_Geo_all.sort_values('Выручка', ascending = False) 

,Гео,Выручка,Абсолютная маржа,Маржа,Средняя маржа,Средний чек,Кол-во чеков,Кол-во товаров,Среднее кол-во товаров в чеке,Кол-во уникальных клиентов,Ценность клиента
2,Регионы,429113227,126684514,0.295224,963.263131,3554,131516,390049,3,85909,1475
0,Москва,276321524,92107371,0.333334,1005.220738,3331,91629,396512,4,61278,1503
1,МО,91347388,28102649,0.307646,941.178506,3405,29859,121761,4,18838,1492


In [103]:
pivot_table_DeliveryType_all = generate_table(df_all, column_name="МетодДоставки") 
pivot_table_DeliveryType_all.sort_values('Выручка', ascending = False) 

,МетодДоставки,Выручка,Абсолютная маржа,Маржа,Средняя маржа,Средний чек,Кол-во чеков,Кол-во товаров,Среднее кол-во товаров в чеке,Кол-во уникальных клиентов,Ценность клиента
0,Магазины,578299551,167130719,0.289004,883.060709,3394,189263,615404,3,124387,1344
2,Курьерская,158091237,56926592,0.360087,1519.014623,4485,37476,220442,6,27620,2061
5,Самовывоз,30196996,11111560,0.367969,605.501608,1647,18351,41038,2,13449,826
1,Pick point,13442935,4798618,0.356962,973.745536,3076,4928,16303,3,4155,1155
3,DPD,11582816,4244530,0.366451,2030.875598,6337,2090,9711,5,1928,2202
4,Транспортная компания,5168605,2682516,0.519002,2993.879464,5947,896,5424,6,734,3655


In [104]:
pivot_table_PaymentType_all = generate_table(df_all, column_name="ФормаОплаты") 
pivot_table_PaymentType_all.sort_values('Выручка', ascending = False) 

,ФормаОплаты,Выручка,Абсолютная маржа,Маржа,Средняя маржа,Средний чек,Кол-во чеков,Кол-во товаров,Среднее кол-во товаров в чеке,Кол-во уникальных клиентов,Ценность клиента
0,Безналичная,608500622,178857533,0.293932,907.082057,3429,197179,646847,3,130471,1371
1,Наличная,188281518,68037002,0.361358,1218.755074,3552,55825,261475,5,40954,1661


In [105]:
pivot_table_WeekDay_all = generate_table(df_all, column_name="ДеньНедели")
pivot_table_WeekDay_all.sort_values('Выручка', ascending = False)

,ДеньНедели,Выручка,Абсолютная маржа,Маржа,Средняя маржа,Средний чек,Кол-во чеков,Кол-во товаров,Среднее кол-во товаров в чеке,Кол-во уникальных клиентов,Ценность клиента
6,Вторник,137471479,42433110,0.308668,932.390903,3332,45510,168884,4,39095,1085
2,Среда,135630202,40450582,0.298242,910.003869,3365,44451,165231,4,38407,1053
0,Четверг,121341291,37035393,0.305217,963.936207,3480,38421,139054,4,33858,1094
1,Воскресенье,112161835,35302711,0.314748,1030.645812,3569,34253,118105,3,30330,1164
4,Понедельник,100994908,32746300,0.324237,1035.063375,3492,31637,108093,3,28334,1156
3,Пятница,98330383,30365366,0.308810,971.567351,3453,31254,111109,4,27875,1089
5,Суббота,90852041,28561072,0.314369,1039.415969,3595,27478,97846,4,24805,1151


In [106]:
pivot_table_TimeHours_all = generate_table(df_all, column_name="ВремяОкругл")
pivot_table_TimeHours_all.sort_values('Выручка', ascending = False)

,ВремяОкругл,Выручка,Абсолютная маржа,Маржа,Средняя маржа,Средний чек,Кол-во чеков,Кол-во товаров,Среднее кол-во товаров в чеке,Кол-во уникальных клиентов,Ценность клиента
14,14,63670854,19912426,0.312740,1031.090824,3623,19312,68373,4,17907,1112
12,17,57923779,18016677,0.311041,1006.574501,3552,17899,66439,4,16607,1085
13,16,57523772,17874446,0.310731,1001.986995,3528,17839,64251,4,16600,1077
2,15,57246953,17802871,0.310984,1013.773191,3567,17561,61694,4,16348,1089
16,19,55466130,17753599,0.320080,1065.450339,3632,16663,59543,4,15506,1145
15,18,52881406,16468794,0.311429,1020.118558,3580,16144,59231,4,15040,1095
11,12,52692673,16448292,0.312155,1033.703620,3606,15912,56865,4,14837,1109
5,13,52407224,16604196,0.316830,1064.849355,3655,15593,56750,4,14565,1140
6,11,48131661,15234613,0.316520,1012.737685,3487,15043,52308,3,14045,1085
17,20,46969434,14477800,0.308239,994.217827,3504,14562,52123,4,13656,1060


In [ ]:
pivot_table_Phone_new_all = generate_table(df_all, column_name="Телефон_new") 
pivot_table_Phone_new_all 

### Сохраняем

In [ ]:
pivot_table_Phone_new_all.to_csv('pivot_table_Phone_new_all.csv', index=False, sep=';', encoding='utf-8-sig')

In [107]:
pivot_table_Month_all.to_csv('pivot_table_Month_all.csv', index=False, sep=';', encoding='utf-8-sig')
pivot_table_Group2_all.to_csv('pivot_table_Group2_all.csv', index=False, sep=';', encoding='utf-8-sig')
pivot_table_CityShop_all.to_csv('pivot_table_CityShop_all.csv', index=False, sep=';', encoding='utf-8-sig')
pivot_table_Geo_all.to_csv('pivot_table_Geo_all.csv', index=False, sep=';', encoding='utf-8-sig')
pivot_table_DeliveryType_all.to_csv('pivot_table_DeliveryType_all.csv', index=False, sep=';', encoding='utf-8-sig')
pivot_table_PaymentType_all.to_csv('pivot_table_PaymentType_all.csv', index=False, sep=';', encoding='utf-8-sig')
pivot_table_WeekDay_all.to_csv('pivot_table_WeekDay_all.csv', index=False, sep=';', encoding='utf-8-sig')
pivot_table_TimeHours_all.to_csv('pivot_table_TimeHours_all.csv', index=False, sep=';', encoding='utf-8-sig')

# По выкупленным товарам. 

### Ориентируемся по 'Новому статусу'. Оставляем только строчки с 'Новым Статусом' == 'Доставлен'. 

В 'Статус' = ‘Доставлен’ есть два подварианта: 'НовыйСтатус' = ‘К отгрузке’ и 'НовыйСтатус' = 'Доставлен'. 
Мы выбираем 'Статус' = ‘Доставлен’: 'НовыйСтатус' = 'Доставлен', значит, должны проверить условие только для 'Нового Статуса'. 

Если мы будем основываться на доставленных товарах, то избежим пропусков в таких полях как 'Дата Доставки', 'Форма оплаты', 'Город доставки'.

In [108]:
df_buyed = df_all[df_all['НовыйСтатус'] == 'Доставлен']

In [109]:
df_buyed.shape

(509572, 45)

**Удалить** все строки, где **есть ПричинаОтмены**, так как тогда товар не может считаться доставленным. 

**Удалить** все строки, где **не Группа2 и Группа3**, так как тогда товар не имеет товарного направлния ("категории") и товарной категории ("подкатегории"). 

In [110]:
# Nan заменим на не сущесвующее значение, чтобы потом удалить все строки не с этим значением 

df_buyed['ПричинаОтмены'] = df_buyed['ПричинаОтмены'].fillna(999999); 
df_buyed = df_buyed[(df_buyed['ПричинаОтмены'] == 999999)] 

<ipython-input-110-7151cf13d660>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_buyed['ПричинаОтмены'] = df_buyed['ПричинаОтмены'].fillna(999999);


In [111]:
df_buyed = df_buyed.dropna(subset = ['Группа2']) 
df_buyed = df_buyed.dropna(subset = ['Группа3']) 

In [112]:
df_buyed.shape

(509572, 45)

In [113]:
df_buyed.to_csv('for_tables_filtered_data_Выкупленные.csv', index=False, sep=';', encoding='utf-8-sig')

### Pivot tables 

In [114]:
pivot_table_Month_buyed = generate_table(df_buyed, column_name="МесяцДатыЗаказа") 
pivot_table_Month_buyed.sort_values('Выручка', ascending = False) 

,МесяцДатыЗаказа,Выручка,Абсолютная маржа,Маржа,Средняя маржа,Средний чек,Кол-во чеков,Кол-во товаров,Среднее кол-во товаров в чеке,Кол-во уникальных клиентов,Ценность клиента
3,8,131725057,35848959,0.272150,804.834965,3072,44542,143864,3,38320,936
0,5,131559550,40288981,0.306241,971.169845,3251,41485,128166,3,34759,1159
2,7,123838443,36163654,0.292023,908.041330,3218,39826,123130,3,34242,1056
1,6,116746415,34118913,0.292248,914.936928,3229,37291,114412,3,32137,1062


In [115]:
pivot_table_Group2_buyed = generate_table(df_buyed, column_name="Группа2") 
pivot_table_Group2_buyed.sort_values('Выручка', ascending = False)

,Группа2,Выручка,Абсолютная маржа,Маржа,Средняя маржа,Средний чек,Кол-во чеков,Кол-во товаров,Среднее кол-во товаров в чеке,Кол-во уникальных клиентов,Ценность клиента
0,КРУПНОГАБАРИТНЫЙ ТОВАР,199015306,54734675,0.275027,1935.660608,7679,28277,33158,1,25572,2140
9,ПОДГУЗНИКИ,79322595,25567431,0.322322,625.196992,2898,40895,58017,1,26683,958
5,ИГРУШКИ,74069279,13459232,0.181711,275.527278,2324,48849,104735,2,41199,327
1,"ТЕКСТИЛЬ, ТРИКОТАЖ",57368930,16007196,0.279022,471.174050,2765,33973,100867,3,25857,619
10,ДЕТСКОЕ ПИТАНИЕ,34340643,22204278,0.646589,1159.976909,3175,19142,76502,4,12650,1755
4,ТОВАРЫ ДЛЯ КОРМЛЕНИЯ,18231143,3238369,0.177628,186.316610,3079,17381,32289,2,14722,220
2,ОБУВЬ,15807185,3996646,0.252837,351.260854,2272,11378,17039,1,9757,410
6,КОСМЕТИКА/ГИГИЕНА,12454732,3279467,0.263311,174.143320,2872,18832,50978,3,14711,223
3,"КАНЦТОВАРЫ, КНИГИ, ДИСКИ",7746400,2457974,0.317305,310.899823,2452,7906,24774,3,7132,345
12,ТОВАРЫ ДЛЯ ЖИВОТНЫХ,4875441,1256662,0.257754,429.334472,2584,2927,8030,3,2078,605


In [116]:
pivot_table_CityShop_buyed = generate_table(df_buyed, column_name="ГородМагазина") 
pivot_table_CityShop_buyed.sort_values('Выручка', ascending = False) 

,ГородМагазина,Выручка,Абсолютная маржа,Маржа,Средняя маржа,Средний чек,Кол-во чеков,Кол-во товаров,Среднее кол-во товаров в чеке,Кол-во уникальных клиентов,Ценность клиента
0,ИнтернетДоставка,455872130,132128636,0.289837,833.761183,2975,158473,504204,3,108608,1217
11,Калуга,4472445,1273598,0.284765,3121.563725,11054,408,469,1,323,3943
2,Ростов-на-Дону,3119593,796734,0.255397,2876.296029,11262,277,314,1,200,3984
5,Москва,2877330,975263,0.338947,2513.564433,7751,388,437,1,370,2636
6,Саратов,2459261,733044,0.298075,3080.016807,10333,238,311,1,210,3491
...,...,...,...,...,...,...,...,...,...,...,...
25,Нижневартовск,152458,51547,0.338106,2577.350000,7623,20,25,1,19,2713
34,Балаково,142590,46681,0.327379,3112.066667,9506,15,19,1,14,3334
60,Казань,118208,33090,0.279930,1946.470588,6953,17,17,1,16,2068
42,Стерлитамак,97292,30993,0.318557,2817.545455,8845,11,11,1,10,3099


In [117]:
pivot_table_Geo_buyed = generate_table(df_buyed, column_name="Гео") 
pivot_table_Geo_buyed.sort_values('Выручка', ascending = False) 

,Гео,Выручка,Абсолютная маржа,Маржа,Средняя маржа,Средний чек,Кол-во чеков,Кол-во товаров,Среднее кол-во товаров в чеке,Кол-во уникальных клиентов,Ценность клиента
2,Регионы,331382983,94618532,0.285526,1008.167455,3654,93852,276075,3,67050,1411
0,Москва,115016457,34659662,0.301345,714.087438,2439,48537,160947,3,33753,1027
1,МО,57470025,17142312,0.298283,825.936497,2837,20755,72550,3,13670,1254


In [118]:
pivot_table_DeliveryType_buyed = generate_table(df_buyed, column_name="МетодДоставки") 
pivot_table_DeliveryType_buyed.sort_values('Выручка', ascending = False) 

,МетодДоставки,Выручка,Абсолютная маржа,Маржа,Средняя маржа,Средний чек,Кол-во чеков,Кол-во товаров,Среднее кол-во товаров в чеке,Кол-во уникальных клиентов,Ценность клиента
0,Магазины,503869465,146420507,0.290592,897.492442,3189,163144,509572,3,112595,1300


In [123]:
pivot_table_PaymentType_buyed = generate_table(df_buyed, column_name="ФормаОплаты") 
pivot_table_PaymentType_buyed.sort_values('Выручка', ascending = False) 

,ФормаОплаты,Выручка,Абсолютная маржа,Маржа,Средняя маржа,Средний чек,Кол-во чеков,Кол-во товаров,Среднее кол-во товаров в чеке,Кол-во уникальных клиентов,Ценность клиента
0,Безналичная,503869465,146420507,0.290592,897.492442,3189,163144,509572,3,112595,1300


In [124]:
pivot_table_WeekDay_buyed = generate_table(df_buyed, column_name="ДеньНедели")
pivot_table_WeekDay_buyed.sort_values('Выручка', ascending = False)

,ДеньНедели,Выручка,Абсолютная маржа,Маржа,Средняя маржа,Средний чек,Кол-во чеков,Кол-во товаров,Среднее кол-во товаров в чеке,Кол-во уникальных клиентов,Ценность клиента
6,Вторник,88366341,25766170,0.291584,851.971365,3026,30243,98053,3,26732,964
2,Среда,84658727,23689618,0.279825,826.862757,3055,28650,93522,3,25650,924
0,Четверг,76193593,21930512,0.287826,888.054748,3184,24695,77751,3,22444,977
1,Воскресенье,71921545,21123082,0.293696,972.517587,3410,21720,65301,3,19855,1064
4,Понедельник,63422422,19119384,0.301461,941.841576,3221,20300,59503,3,18695,1023
3,Пятница,61174002,17528119,0.286529,884.276006,3187,19822,61135,3,18245,961
5,Суббота,58132835,17263622,0.296969,974.575025,3382,17714,54307,3,16388,1053


In [125]:
pivot_table_TimeHours_buyed = generate_table(df_buyed, column_name="ВремяОкругл")
pivot_table_TimeHours_buyed.sort_values('Выручка', ascending = False)

,ВремяОкругл,Выручка,Абсолютная маржа,Маржа,Средняя маржа,Средний чек,Кол-во чеков,Кол-во товаров,Среднее кол-во товаров в чеке,Кол-во уникальных клиентов,Ценность клиента
12,14,38946214,11377747,0.292140,946.883073,3345,12016,36522,3,11364,1001
13,15,35514645,10394343,0.292678,934.406958,3288,11124,33236,3,10567,984
11,16,34925284,10197127,0.291970,913.311867,3225,11165,34838,3,10584,963
10,17,34166984,9871666,0.288924,907.405644,3249,10879,34033,3,10300,958
15,19,33982178,10031799,0.295208,964.317889,3377,10403,31475,3,9892,1014
8,12,31631170,9282298,0.293454,954.085518,3347,9729,28844,3,9237,1005
14,18,31536143,9232680,0.292765,930.995261,3276,9917,31138,3,9441,978
2,13,30390159,8841073,0.290919,935.860379,3323,9447,28422,3,9009,981
9,11,30219160,8844501,0.292679,930.510363,3275,9505,27345,3,9003,982
16,20,29224201,8400589,0.287453,906.603605,3254,9266,28953,3,8882,946


In [ ]:
pivot_table_Phone_new_buyed = generate_table(df_buyed, column_name="Телефон_new") 
pivot_table_Phone_new_buyed

### Сохраняем

In [ ]:
pivot_table_Phone_new_buyed.to_csv('pivot_table_Phone_new_buyed.csv', index=False, sep=';', encoding='utf-8-sig')

In [126]:
pivot_table_Month_buyed.to_csv('pivot_table_Month_buyed.csv', index=False, sep=';', encoding='utf-8-sig')
pivot_table_Group2_buyed.to_csv('pivot_table_Group2_buyed.csv', index=False, sep=';', encoding='utf-8-sig')
pivot_table_CityShop_buyed.to_csv('pivot_table_CityShop_buyed.csv', index=False, sep=';', encoding='utf-8-sig')
pivot_table_Geo_buyed.to_csv('pivot_table_Geo_buyed.csv', index=False, sep=';', encoding='utf-8-sig')
pivot_table_DeliveryType_buyed.to_csv('pivot_table_DeliveryType_buyed.csv', index=False, sep=';', encoding='utf-8-sig')
pivot_table_PaymentType_buyed.to_csv('pivot_table_PaymentType_buyed.csv', index=False, sep=';', encoding='utf-8-sig')
pivot_table_WeekDay_buyed.to_csv('pivot_table_WeekDay_buyed.csv', index=False, sep=';', encoding='utf-8-sig')
pivot_table_TimeHours_buyed.to_csv('pivot_table_TimeHours_buyed.csv', index=False, sep=';', encoding='utf-8-sig')